Preprocessing Data

In [1]:
import pandas as pd
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
nltk.download('punkt_tab')
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ujjwa_n18433z\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ujjwa_n18433z\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ujjwa_n18433z\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Load file

In [6]:
ds = pd.read_csv("C:\\Users\\ujjwa_n18433z\\Desktop\\ujjwal\\All Projects\\Project_WoC_7.0_Fake_Review_Detection\\fakeReviewData.csv")


Removing Digit , Punctuation and Extra Space 

In [11]:
ds["text_"] = ds["text_"].str.replace(r'\d+', "", regex=True).str.lower()
ds["text_"] = ds["text_"].str.translate(str.maketrans('', '', string.punctuation)).str.strip().str.replace(r'\s+',' ', regex=True)

Removing Stopwords

In [12]:
ds["text_"] = ds["text_"].apply(lambda x: " ".join(word for word in word_tokenize(x) if word not in stopwords))

Lemmatizing Words

In [13]:
lemmatizer = WordNetLemmatizer()
ds["text_"] = ds["text_"].apply(lambda x : "".join(lemmatizer.lemmatize(word) for word in x))

Vectorizing the review 


In [14]:
tfidf_vectorizer=TfidfVectorizer(max_features=500)
tfidf_vector = tfidf_vectorizer.fit_transform(ds["text_"])

In [15]:
dfidf_df=pd.DataFrame(tfidf_vector.toarray(),columns=tfidf_vectorizer.get_feature_names_out())

In [17]:
ds=pd.concat([ds,dfidf_df],axis=1)

In [18]:
ds

,category,rating,label,text_,able,absolutely,acting,action,actually,add,...,would,writing,written,wrong,year,years,yet,youll,young,youre
0,Home_and_Kitchen_5,5.0,CG,love well made sturdy comfortable love itvery ...,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1,Home_and_Kitchen_5,5.0,CG,love great upgrade original ive mine couple years,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.429135,0.0,0.0,0.0,0.000000
2,Home_and_Kitchen_5,5.0,CG,pillow saved back love look feel pillow,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
3,Home_and_Kitchen_5,1.0,CG,missing information use great product price,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
4,Home_and_Kitchen_5,5.0,CG,nice set good quality set two months,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40427,Clothing_Shoes_and_Jewelry_5,4.0,OR,read reviews saying bra ran small ordered two ...,0.172311,0.0,0.0,0.0,0.094095,0.0,...,0.051699,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
40428,Clothing_Shoes_and_Jewelry_5,5.0,CG,wasnt sure exactly would little large small si...,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.205517,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
40429,Clothing_Shoes_and_Jewelry_5,2.0,OR,wear hood wear hood wear jacket without hood s...,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.051273,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.291747
40430,Clothing_Shoes_and_Jewelry_5,1.0,CG,liked nothing dress reason gave stars ordered ...,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.039123,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
